<a href="https://colab.research.google.com/github/salu133445/ismir2019tutorial/blob/main/musegan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generating Music with GANs

## Prerequisites

### Install dependencies

In [1]:
!pip3 install pypianoroll livelossplot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.9 MB/s eta 0:00:00
  Created wheel for pretty-midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592305 sha256=9ae93ef1b01db916194cdbbd34eed8eae1561ff17e23733ea306ba4fcbf127b0
  Stored in directory: /root/.cache/pip/wheels/ea/8a/5a/615ed5b0cab54051df02fbaa4ad5334f91e3156b8c8753f5fc
Successfully built pretty-midi


### Import libraries

In [2]:
from IPython.display import clear_output
from ipywidgets import interact, IntSlider

import os
import os.path
import random
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import pypianoroll
from pypianoroll import Multitrack, Track, BinaryTrack
from tqdm import tqdm
from livelossplot import PlotLosses
from livelossplot.outputs import MatplotlibPlot

## Settings

In [3]:
# Data
n_tracks = 4  # number of tracks
n_pitches = 72  # number of pitches
lowest_pitch = 24  # MIDI note number of the lowest pitch
n_samples_per_song = 4  # number of samples to extract from each song in the datset
n_measures = 4  # number of measures per sample
beat_resolution = 4  # temporal resolution of a beat (in timestep)
programs = [0, 0, 25, 33]  # program number for each track
is_drums = [True, False, False, False]  # drum indicator for each track
track_names = ['Drums', 'Piano', 'Guitar', 'Bass']  # name of each track
tempo = 70

# Training
batch_size = 16
latent_dim = 128
n_steps = 50000

# Sampling
sample_interval = 500  # interval to run the sampler (in step)
n_samples = 4

In [4]:
measure_resolution = 4 * beat_resolution
tempo_array = np.full((4 * 4 * measure_resolution, 1), tempo)
assert 24 % beat_resolution == 0, (
    "beat_resolution must be a factor of 24 (the beat resolution used in "
    "the source dataset)."
)
assert len(programs) == len(is_drums) and len(programs) == len(track_names), (
    "Lengths of programs, is_drums and track_names must be the same."
)    

## Data Prepration

In [5]:
data = []
root_dir = '/kaggle/input/cleaned-musegan-data/cleaned_data'
song_list = os.listdir(root_dir)
print(len(song_list))
for idx in tqdm(range(len(song_list))):
    multitrack = pypianoroll.load(os.path.join(root_dir,song_list[idx]))
    for track in multitrack.tracks:
        track_name = track.name.split(' ')
        track_name = [name.lower() for name in track_name]
        if 'drums' in track_name or 'bass' in track_name or 'guitar' in track_name or 'piano' in track_name:
            continue
        multitrack.tracks.remove(track)
    if len(multitrack.tracks) < 4:
        continue
    multitrack.tracks = multitrack.tracks[0:4]
    multitrack.binarize()
    multitrack.set_resolution(beat_resolution)
    pianoroll= (multitrack.stack() > 0)
    pianoroll = pianoroll[:, :, lowest_pitch: lowest_pitch + n_pitches]
    n_total_measures = multitrack.get_max_length() // (4 * n_measures)
    candidate = n_total_measures - n_measures
    target_n_samples = min(n_total_measures // n_measures, n_samples_per_song)
    if target_n_samples > 0:
        for i in np.random.choice(candidate, target_n_samples, False):
            start = i * 4 * beat_resolution
            end = (i + n_measures) * 4 * beat_resolution
            if (pianoroll.sum(axis=(1, 2)) < 10).any():
                continue
    else:
        continue
    data.append(torch.as_tensor(pianoroll[:, start:end], dtype=torch.float32))

14410


100%|██████████| 14410/14410 [09:48<00:00, 24.48it/s]


In [6]:
class MuseGANDataset(Dataset):

    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return data[idx]


dataset = MuseGANDataset(data)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

## Model

 ### Define the generator

In [7]:
class GeneraterBlock(torch.nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()
        self.transconv = torch.nn.ConvTranspose3d(in_dim, out_dim, kernel, stride)
        self.batchnorm = torch.nn.BatchNorm3d(out_dim)
    
    def forward(self, x):
        x = self.transconv(x)
        x = self.batchnorm(x)
        return torch.nn.functional.relu(x)

In [8]:

class Generator(torch.nn.Module):
    """A convolutional neural network (CNN) based generator. The generator takes
    as input a latent vector and outputs a fake sample."""
    def __init__(self):
        super().__init__()
        self.transconv0 = torch.nn.Sequential(
            GeneraterBlock(128, 2048, (2, 1, 1), (2, 1, 1)),
            GeneraterBlock(2048, 1024, (2, 1, 1), (2, 1, 1)),
            GeneraterBlock(1024, 512, (1, 2, 1), (1, 2, 1)),
            GeneraterBlock(512, 256, (1, 2, 1), (1, 2, 1)),
            GeneraterBlock(256, 128, (1, 1, 2), (1, 1, 2)),
            GeneraterBlock(128, 64, (1, 1, 2), (1, 1, 2)),
            GeneraterBlock(64, 32, (1, 1, 3), (1, 1, 1))
        )
        self.transconv1 = torch.nn.ModuleList([
            GeneraterBlock(32, 16, (1, 4, 1), (1, 4, 1))
            for _ in range(4)
        ])
        self.transconv2 = torch.nn.ModuleList([
            torch.nn.Sequential(
                torch.nn.ConvTranspose3d(16, 1, (1, 1, 12), (1, 1, 12)),
                torch.nn.Tanh()
            )
            for _ in range(4)
        ])

    def forward(self, x):
        x = x.view(-1, 128, 1, 1, 1)
        x = self.transconv0(x)
        x = [transconv(x) for transconv in self.transconv1]
        x = torch.cat([transconv(x_) for x_, transconv in zip(x, self.transconv2)], 1)
        x = x.view(-1, 4, 4 * 16, 72)
        return x

 ### Define the discriminator

In [9]:
class LayerNorm(torch.nn.Module):
    """An implementation of Layer normalization that does not require size
    information. Copied from https://github.com/pytorch/pytorch/issues/1959."""
    def __init__(self, n_features, eps=1e-5, affine=True):
        super().__init__()
        self.n_features = n_features
        self.affine = affine
        self.eps = eps
        if self.affine:
            self.gamma = torch.nn.Parameter(torch.Tensor(n_features).uniform_())
            self.beta = torch.nn.Parameter(torch.zeros(n_features))

    def forward(self, x):
        shape = [-1] + [1] * (x.dim() - 1)
        mean = x.view(x.size(0), -1).mean(1).view(*shape)
        std = x.view(x.size(0), -1).std(1).view(*shape)
        y = (x - mean) / (std + self.eps)
        if self.affine:
            shape = [1, -1] + [1] * (x.dim() - 2)
            y = self.gamma.view(*shape) * y + self.beta.view(*shape)
        return y

In [10]:
class DiscriminatorBlock(torch.nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride):
        super().__init__()
        self.transconv = torch.nn.Conv3d(in_dim, out_dim, kernel, stride)
        self.layernorm = LayerNorm(out_dim)
        self.dropout = torch.nn.Dropout(p=0.1)
    def forward(self, x):
        x = self.transconv(x)
        x = self.layernorm(x)
        x = self.dropout(x)
        return torch.nn.functional.leaky_relu(x)

In [11]:
class Discriminator(torch.nn.Module):
    """A convolutional neural network (CNN) based discriminator. The
    discriminator takes as input either a real sample (in the training data) or
    a fake sample (generated by the generator) and outputs a scalar indicating
    its authentity.
    """
    def __init__(self):
        super().__init__()
        self.conv0 = torch.nn.ModuleList([
            DiscriminatorBlock(1, 16, (1, 1, 12), (1, 1, 12)) for _ in range(4)
        ])
        self.conv1 = torch.nn.ModuleList([
            DiscriminatorBlock(16, 16, (1, 4, 1), (1, 4, 1)) for _ in range(4)
        ])
        self.conv2 = torch.nn.Sequential(
            DiscriminatorBlock(16 * 4, 64, (1, 1, 3), (1, 1, 1)),
            DiscriminatorBlock(64, 128, (1, 1, 2), (1, 1, 2)),
            DiscriminatorBlock(128, 128, (1, 1, 2), (1, 1, 2)),
            DiscriminatorBlock(128, 256, (1, 2, 1), (1, 2, 1)),
            DiscriminatorBlock(256, 256, (1, 2, 1), (1, 2, 1)),
            DiscriminatorBlock(256, 512, (2, 1, 1), (1, 1, 1)),
            DiscriminatorBlock(512, 512, (3, 1, 1), (3, 1, 1))
        )
        self.dense = torch.nn.Linear(512, 1)

    def forward(self, x):
        #print('d_input', x.shape)
        x = x.view(-1, 4, 4, 16, 72)
        x = [conv(x[:, [i]]) for i, conv in enumerate(self.conv0)]
        x = torch.cat([conv(x_) for x_, conv in zip(x, self.conv1)], 1)
        x = self.conv2(x)
        x = x.view(-1, 512)
        x = self.dense(x)
        return x
     


## Training functions

In [12]:
def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    """Compute the gradient penalty for regularization. Intuitively, the
    gradient penalty help stablize the magnitude of the gradients that the
    discriminator provides to the generator, and thus help stablize the training
    of the generator."""
    # Get random interpolations between real and fake samples
    alpha = torch.rand(real_samples.size(0), 1, 1, 1).cuda()
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples))
    interpolates = interpolates.requires_grad_(True)
    # Get the discriminator output for the interpolations
    d_interpolates = discriminator(interpolates)
    # Get gradients w.r.t. the interpolations
    fake = torch.ones(real_samples.size(0), 1).cuda()
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,  
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    # Compute gradient penalty
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [13]:
def train_one_step(d_optimizer, g_optimizer, real_samples):
    """Train the networks for one step."""
    # Sample from the lantent distribution
    latent = torch.rand(batch_size, latent_dim)

    # Transfer data to GPU
    if torch.cuda.is_available():
        real_samples = real_samples .cuda()
        latent = latent.cuda()
    
    # === Train the discriminator ===
    for _ in range(5):
        # Reset cached gradients to zero
        d_optimizer.zero_grad()
        # Get discriminator outputs for the real samples
        prediction_real = discriminator(real_samples)
        # Compute the loss function
        # d_loss_real = torch.mean(torch.nn.functional.relu(1. - prediction_real))
        d_loss_real = -torch.mean(prediction_real)
        # Backpropagate the gradients
        #d_loss_real.backward()
        #print('d_loss_real', d_loss_real.shape)
        
        # Generate fake samples with the generator
        fake_samples = generator(latent)
        # Get discriminator outputs for the fake samples
        prediction_fake_d = discriminator(fake_samples.detach())
        # Compute the loss function
        # d_loss_fake = torch.mean(torch.nn.functional.relu(1. + prediction_fake_d))
        d_loss_fake = torch.mean(prediction_fake_d)
        # Backpropagate the gradients
        #d_loss_fake.backward()
        #print('d_loss_fake', d_loss_fake.shape)

        # Compute gradient penalty
        gradient_penalty = 10.0 * compute_gradient_penalty(
            discriminator, real_samples.data, fake_samples.data
        )
        # Backpropagate the gradients
        #gradient_penalty.backward()
        #print('gradient_penalty', gradient_penalty.shape)
        d_loss = d_loss_real + d_loss_fake + gradient_penalty
        d_loss.backward()

        # Update the weights
        d_optimizer.step()
    
    # === Train the generator ===
    # Reset cached gradients to zero
    g_optimizer.zero_grad()
    # Get discriminator outputs for the fake samples
    prediction_fake_g = discriminator(fake_samples)
    # Compute the loss function
    g_loss = -torch.mean(prediction_fake_g)
    # Backpropagate the gradients
    g_loss.backward()
    # Update the weights
    g_optimizer.step()

    return d_loss, g_loss

## Training Preparation

In [14]:
# Create data loader
# data_loader = get_data_loader()

# Create neural networks
discriminator = Discriminator()
generator = Generator()
print("Number of parameters in G: {}".format(
    sum(p.numel() for p in generator.parameters() if p.requires_grad)))
print("Number of parameters in D: {}".format(
    sum(p.numel() for p in discriminator.parameters() if p.requires_grad)))

# Create optimizers
d_optimizer = torch.optim.Adam(
    discriminator.parameters(), lr=0.001,  betas=(0.5, 0.9))
g_optimizer = torch.optim.Adam(
    generator.parameters(), lr=0.001, betas=(0.5, 0.9))

# Prepare the inputs for the sampler, which wil run during the training
sample_latent = torch.rand(n_samples, latent_dim)

# Transfer the neural nets and samples to GPU
if torch.cuda.is_available():
    discriminator = discriminator.cuda()
    generator = generator.cuda()
    sample_latent = sample_latent.cuda()

# Create an empty dictionary to sotre history samples
history_samples = {}

# Create a LiveLoss logger instance for monitoring
liveloss = PlotLosses(outputs=[MatplotlibPlot(cell_size=(6,2))])

# Initialize step
step = 0

Number of parameters in G: 6138724
Number of parameters in D: 1317953


## Training

In [15]:
print(len(dataset))

14243


In [ ]:
# Create a progress bar instance for monitoring
progress_bar = tqdm(total=n_steps, initial=step, ncols=80, mininterval=1)
best_d_loss = 10000.0
best_g_loss = 10000.0
# Start iterations
while step < n_steps + 1:
    # Iterate over the dataset
    for real_samples in data_loader:
        # Train the neural networks
        generator.train()
        d_loss, g_loss = train_one_step(d_optimizer, g_optimizer, real_samples)
        if abs(d_loss) < best_d_loss :
            best_d_loss = abs(d_loss)
            generator.eval()
            with torch.no_grad():
                torch.save(generator, f'generator.pt') 
        if step % 500 == 0:
            # Record smoothened loss values to LiveLoss logger
            liveloss.update({'d_loss': d_loss, 'g_loss': g_loss})
            
            # Update losses to progress bar
            progress_bar.set_description_str(
                "(d_loss={: 8.6f}, g_loss={: 8.6f})".format(d_loss, g_loss)
            )

            # Get generated samples
            generator.eval()
            sample_latent = torch.rand(n_samples, latent_dim).cuda()
            with torch.no_grad():
                torch.save(generator, f'generator_{step}.pt')
                samples = generator(sample_latent).cpu().detach().numpy()
            history_samples[step] = samples

            # Display loss curves
            clear_output(True)
            
            # Display generated samples
            samples = samples.transpose(1, 0, 2, 3).reshape(n_tracks, -1, n_pitches)
            tracks = []
            for idx, (program, is_drum, track_name) in enumerate(
                zip(programs, is_drums, track_names)
            ):
                pianoroll = np.pad(
                    samples[idx] > 0.5,
                    ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))
                )
                tracks.append(
                    BinaryTrack(
                        name=track_name,
                        program=program,
                        is_drum=is_drum,
                        pianoroll=pianoroll
                    )
                )
            m = Multitrack(
                tracks=tracks,
                tempo=tempo_array,
                resolution=beat_resolution
            )
            m.write(f'./train_v3_{step}.mid') # download generated melody
            plt.show()
            
        step += 1
        progress_bar.update(1)
        if step >= n_steps:
            break

(d_loss=-6.820445, g_loss=-1.531428):  13%|▏| 6417/50000 [45:38<5:02:30,  2.40it

## Training History

In [ ]:
# Show history
steps = [5000, 10000, 15000, 20000]
for step in steps:
    print(f"Step={step}")
    samples = history_samples[step].transpose(1, 0, 2, 3).reshape(n_tracks, -1, n_pitches)
    tracks = []
    for idx, (program, is_drum, track_name) in enumerate(zip(programs, is_drums, track_names)):
        pianoroll = np.pad(
            samples[idx] > 0.5,
            ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))
        )
        tracks.append(
            BinaryTrack(
                name=track_name,
                program=program,
                is_drum=is_drum,
                pianoroll=pianoroll,
            )
        )
    m = Multitrack(tracks=tracks, tempo=tempo_array, resolution=beat_resolution)

    axs = m.plot()
    for ax in axs:
        for x in range(
            measure_resolution,
            4 * measure_resolution * n_measures,
            measure_resolution
        ):
            if x % (measure_resolution * 4) == 0:
                ax.axvline(x - 0.5, color='k')
            else:
                ax.axvline(x - 0.5, color='k', linestyle='-', linewidth=1)
    plt.gcf().set_size_inches((16, 8))
    plt.show()
print(best_d_loss, best_g_loss)

# Generating a new melody

In [ ]:
with torch.no_grad():
    generated = generator(torch.rand(batch_size, latent_dim).cuda()).cpu().detach().numpy()
    generated = generated.transpose(1, 0, 2, 3).reshape(n_tracks, -1, n_pitches)

tracks = []
for idx, (program, is_drum, track_name) in enumerate(zip(programs, is_drums, track_names)):
    pianoroll = np.pad(
        generated[idx] > 0.5,
        ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))
    )
    tracks.append(
        BinaryTrack(
            name=track_name,
            program=program,
            is_drum=is_drum,
            pianoroll=pianoroll,
        )
    )
m = Multitrack(tracks=tracks, tempo=tempo_array, resolution=beat_resolution)

m.write(f'./train_v3_generated.mid')

axs = m.plot()
plt.gcf().set_size_inches((16, 8))
plt.show()